In [3]:
# Imports
#%pip install duckdb
#%pip install sqlite
import tensorflow as tf
import keras
import pandas as pd
import numpy as np 
import sys
import re
sys.path.insert(0, "modules")
import seaborn as sns
import pickle
import os
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor



Using TensorFlow backend.


In [27]:
diamonds = pd.read_csv('../data/diamonds_train_data/diamonds.csv')

In [28]:
diamonds.head()

,cut,color,clarity,price,carat,city,depth,table,x,y,z
0,Very Good,H,VS2,505,0.32,Kimberly,63.0,57.0,4.35,4.38,2.75
1,Very Good,G,VS1,605,0.30,Zurich,61.7,56.0,4.29,4.33,2.66
2,Very Good,H,SI1,7066,1.55,Antwerp,63.2,57.0,7.35,7.31,4.64
3,Very Good,H,VS1,2422,0.70,Kimberly,62.1,56.0,5.65,5.68,3.52
4,Very Good,H,SI1,2553,0.80,Kimberly,63.2,54.0,5.93,5.88,3.73


In [29]:
diamonds_df=diamonds
diamonds_df.head()

,cut,color,clarity,price,carat,city,depth,table,x,y,z
0,Very Good,H,VS2,505,0.32,Kimberly,63.0,57.0,4.35,4.38,2.75
1,Very Good,G,VS1,605,0.30,Zurich,61.7,56.0,4.29,4.33,2.66
2,Very Good,H,SI1,7066,1.55,Antwerp,63.2,57.0,7.35,7.31,4.64
3,Very Good,H,VS1,2422,0.70,Kimberly,62.1,56.0,5.65,5.68,3.52
4,Very Good,H,SI1,2553,0.80,Kimberly,63.2,54.0,5.93,5.88,3.73


In [31]:
# Assuming diamonds is already loaded as diamonds_df
diamonds_df = diamonds

# Drop 'city' column if it exists
if 'city' in diamonds_df.columns:
    diamonds_df = diamonds_df.drop(columns=['city'])

# Function to adjust outliers
def adjust_outliers(df):
    df_adjusted = df.copy()
    for col in df.columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_limit = Q1 - 1.75 * IQR
        upper_limit = Q3 + 1.67 * IQR
        df_adjusted[col] = df_adjusted[col].clip(lower_limit, upper_limit)
    return df_adjusted

# Adjust outliers for specific columns
outlier_columns = ['depth', 'table', 'x', 'y', 'z', 'carat']
diamonds_df[outlier_columns] = adjust_outliers(diamonds_df[outlier_columns])

# Function to label categorical data
def data_labeling(df, cols, dictio):
    for col in cols:
        df[col] = df[col].replace(dictio)
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

# Dictionary for labeling
dictio = {
    'Fair': 1.0,
    'Good': 1.25,
    'Very Good': 1.75,
    'Premium': 2.0,
    'Ideal': 2.5,
    'J': 1.0,
    'I': 1.5,
    'H': 1.75,
    'G': 2.0,
    'F': 2.25,
    'E': 2.5,
    'D': 2.75,
    'I1': 1.0,
    'SI2': 1.25,
    'SI1': 1.75,
    'VS2': 2.0,
    'VS1': 2.25,
    'VVS2': 2.5,
    'VVS1': 2.75,
    'IF': 3.0
}

# Label categorical columns
diamonds_df = data_labeling(diamonds_df, ['cut', 'color', 'clarity'], dictio)

# Fill missing values with column means
if diamonds_df.isnull().sum().sum() > 0:
    diamonds_df.fillna(diamonds_df.mean(numeric_only=True), inplace=True)

# Feature engineering
diamonds_df['carat_depth'] = diamonds_df['carat'] * diamonds_df['depth']
diamonds_df['carat_table'] = diamonds_df['carat'] * diamonds_df['table']
diamonds_df['carat_depth_table'] = diamonds_df['carat'] * diamonds_df['depth'] * diamonds_df['table']
diamonds_df['log_carat'] = np.log1p(diamonds_df['carat'])
diamonds_df['log_y'] = np.log1p(diamonds_df['y'])
diamonds_df['log_x_y'] = np.log1p(diamonds_df['y'] / diamonds_df['x']) ** 9
#diamonds_df['x_y_radial'] = np.sqrt(diamonds_df['log_x/y'])
#diamonds_df['y'] = np.log1p(diamonds_df['y'])**2
#diamonds_df['volum'] = np.log1p(diamonds_df['y']*diamonds_df['x']*diamonds_df['z'])
#diamonds_df['carat'] = (diamonds_df['carat'])**15

# Split data into features and target
X = diamonds_df.drop(columns=['price'])
y = diamonds_df['price']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize XGBRegressor
model = XGBRegressor()

# Grid search parameters
param_grid = {
    'colsample_bytree': [0.85],
    'learning_rate': [0.1],
    'max_depth': [6],
    'n_estimators': [150],
    'subsample': [0.8],
    'gamma': [0]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=3, scoring='neg_root_mean_squared_error')

# Fit the GridSearchCV instance to the training data
grid_search.fit(X, y)

# Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best parameters found: ", best_params)
print("Best score found: ", best_score)

# Retrieve the best model
best_model = grid_search.best_estimator_

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   16.6s remaining:   25.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   16.9s finished


Best parameters found:  {'colsample_bytree': 0.85, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 150, 'subsample': 0.8}
Best score found:  -523.2947475593915


In [ ]:
print("Best score found: ", best_score)

In [14]:
filename = '../models/XGB0014.sav'
pickle.dump(grid_search, open(filename, 'wb'))
print('Your model has been saved with  pickle!!!')

Your model has been saved with  pickle!!!


## Test

---

In [15]:
diamonds_test= pd.read_csv('../data/diamonds_test_data/diamonds_test.csv')

In [16]:
diamonds_test

,id,carat,cut,color,clarity,depth,table,x,y,z,city
0,0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam
...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam
13481,13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City
13482,13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv
13483,13483,0.70,Very Good,F,SI2,58.8,57.0,5.85,5.89,3.45,Surat


In [17]:

if 'city' in diamonds_test.columns:
    diamonds_test= diamonds_test.drop(columns=['city'])
    
outlier_columns = ['depth','table', 'x', 'y', 'z','carat']
diamonds_test[outlier_columns] = adjust_outliers(diamonds_test[outlier_columns])


dictio = {
    'Fair': 1.0,
    'Good': 1.25,
    'Very Good': 1.75,
    'Premium': 2.0,
    'Ideal': 2.5,
    'J': 1.0,
    'I': 1.5,
    'H': 1.75,
    'G': 2.0,
    'F': 2.25,
    'E': 2.5,
    'D': 2.75,
    'I1': 1.0,
    'SI2': 1.25,
    'SI1': 1.75,
    'VS2': 2.0,
    'VS1': 2.25,
    'VVS2': 2.5,
    'VVS1': 2.75,
    'IF': 3
}

data_labeling(diamonds_test, ['cut', 'color', 'clarity'],dictio)
   
    


# Verificación y relleno de valores faltantes
if diamonds_test.isnull().sum().sum() > 0:
    ddiamonds_test.fillna(diamonds_test.mean(numeric_only=True), inplace=True)

# Ingeniería de características
diamonds_test['carat_depth'] = diamonds_test['carat'] * diamonds_test['depth']
diamonds_test['carat_table'] = diamonds_test['carat'] * diamonds_test['table']
diamonds_test['carat_depth_table'] = diamonds_test['carat'] * diamonds_test['depth'] * diamonds_test['table']
diamonds_test['log_carat'] = np.log1p(diamonds_test['carat'])
diamonds_test['log_y'] = np.log1p(diamonds_test['y'])
diamonds_test['log_x/y'] = np.log1p(diamonds_test['y']/diamonds_test['x'])**9


    
diamonds_test

,id,carat,cut,color,clarity,depth,table,x,y,z,carat_depth,carat_table,carat_depth_table,log_carat,log_y,log_x/y
0,0,0.79,1.75,2.25,1.75,62.7,60.0,5.82,5.89,3.67,49.533,47.40,2971.980,0.582216,1.930071,0.039911
1,1,1.20,2.50,1.00,2.25,61.0,57.0,6.81,6.89,4.18,73.200,68.40,4172.400,0.788457,2.065596,0.039840
2,2,1.57,2.00,1.75,1.75,62.2,61.0,7.38,7.32,4.57,97.654,95.77,5956.894,0.943906,2.118662,0.035026
3,3,0.90,1.75,2.25,1.75,63.8,54.0,6.09,6.13,3.90,57.420,48.60,3100.680,0.641854,1.964311,0.038537
4,4,0.50,1.75,2.25,2.25,62.9,58.0,5.05,5.09,3.19,31.450,29.00,1824.100,0.405465,1.806648,0.038874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.57,2.50,2.50,1.75,61.9,56.0,5.35,5.32,3.30,35.283,31.92,1975.848,0.451076,1.843719,0.035609
13481,13481,0.71,2.50,1.50,2.00,62.2,55.0,5.71,5.73,3.56,44.162,39.05,2428.910,0.536493,1.906575,0.037782
13482,13482,0.70,2.50,2.25,2.25,61.6,55.0,5.75,5.71,3.53,43.120,38.50,2371.600,0.530628,1.903599,0.035296
13483,13483,0.70,1.75,2.25,1.25,58.8,57.0,5.85,5.89,3.45,41.160,39.90,2346.120,0.530628,1.930071,0.038604


In [18]:
neworder = diamonds_df.columns.tolist()
neworder.remove( 'price')

diamonds_test = diamonds_test.reindex(columns=neworder)
diamonds_test

,cut,color,clarity,carat,depth,table,x,y,z,carat_depth,carat_table,carat_depth_table,log_carat,log_y,log_x/y
0,1.75,2.25,1.75,0.79,62.7,60.0,5.82,5.89,3.67,49.533,47.40,2971.980,0.582216,1.930071,0.039911
1,2.50,1.00,2.25,1.20,61.0,57.0,6.81,6.89,4.18,73.200,68.40,4172.400,0.788457,2.065596,0.039840
2,2.00,1.75,1.75,1.57,62.2,61.0,7.38,7.32,4.57,97.654,95.77,5956.894,0.943906,2.118662,0.035026
3,1.75,2.25,1.75,0.90,63.8,54.0,6.09,6.13,3.90,57.420,48.60,3100.680,0.641854,1.964311,0.038537
4,1.75,2.25,2.25,0.50,62.9,58.0,5.05,5.09,3.19,31.450,29.00,1824.100,0.405465,1.806648,0.038874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,2.50,2.50,1.75,0.57,61.9,56.0,5.35,5.32,3.30,35.283,31.92,1975.848,0.451076,1.843719,0.035609
13481,2.50,1.50,2.00,0.71,62.2,55.0,5.71,5.73,3.56,44.162,39.05,2428.910,0.536493,1.906575,0.037782
13482,2.50,2.25,2.25,0.70,61.6,55.0,5.75,5.71,3.53,43.120,38.50,2371.600,0.530628,1.903599,0.035296
13483,1.75,2.25,1.25,0.70,58.8,57.0,5.85,5.89,3.45,41.160,39.90,2346.120,0.530628,1.930071,0.038604


In [19]:
filename = '../models/XGB0014.sav'
model = pickle.load(open(filename, 'rb'))


In [20]:
predictions=model.predict(diamonds_test)


In [21]:


df_predictions = pd.DataFrame(predictions, columns=['price'], index=range(len(predictions)))
df_predictions = df_predictions.rename_axis('id')
df_predictions

,price
id,
0,2982.814209
1,5689.636719
2,9465.033203
3,4061.862549
4,1667.564331
...,...
13480,1685.923218
13481,2455.858887
13482,3110.600586


In [22]:
df_predictions.to_csv('../data/submission/006.csv')